In [65]:
from defisurl import DefisUrl # Importation du code fournit par pydefis.fr pour récuperer l'image et renvoyer le resultat.

from time import sleep # Importation de la fonction 'sleep' dans la librairie 'time' pour ajouter un délai.

import json # Importation de la librairie Json pour ouvrir notre dictionnaire référence.

# Importation des librairies pour transformer l'image de la base 64 vers un PNG:
from PIL import Image
import base64
from io import BytesIO

#----Fonction qui décompose l'image en matrices (une par alien)------------
#---Chaque alien sera transformé en une matrice (avec 1 valeur par pixel)------
#---On met '0' pour un pixel noir, '1' pour un pixel coloré------
def decomposition(image_pil): # On récupère l'image à tester.

    liste_matrice_alien = [] # La liste qui contiendra les matrices de chaque alien.

    for y in range(0,4) : # On itére les lignes d'aliens dans l'image original.
        for x in range(0,6) : # On itére les colonnes d'aliens dans l'image original.

            matrice_alien = [] # Matrice d'un seul alien initialisé.

            for y_alien in range (y*64 , (64*y)+64) : # On parcourt l'image original, sur l'axe y, de 64 en 64 (taille d'un cube contenant 1 alien).
                liste_pixel = [] # Ligne d'une matrice d'un seul alien.
                for x_alien in range (x*64 , (x*64)+64) : # On parcourt l'image original, sur l'axe x, de 64 en 64 (taille d'un cube contenant 1 alien).

                    r, g, b = image_pil.getpixel((x_alien, y_alien)) # On récupère les valeurs r, g, b de l'image.

                    if r == 0 and g == 0 and b == 0 : # Si le pixel est noir.
                        liste_pixel.append(0) # On stock '0' dans la matrice.

                    else : # Sinon (= pixel non-noir).
                        liste_pixel.append(1) # On stock '1' dans la matrice.

                matrice_alien.append(liste_pixel) # Quand la ligne est remplit, on l'ajoute à la matrice du alien.
            liste_matrice_alien.append(matrice_alien) # On ajoute la matrice complète d'un alien dans notre liste.
            
    return(liste_matrice_alien) # On revoit la liste des 24 matrices.


#----Fonction qui génère le resultat----
#---Chaque matrice des aliens sera comparé avec nos matrices références------
#---Le résultat sera une chaine de caractère, de A à I suivant l'alien détecté------
def comparer(reference, liste_matrice_alien): # On récupère le dictionnaire contenant les matrices références ainsi que la liste des matrices de aliens à traiter.
    resultat = '' # Déclaration du résultat.

    for matrice_alien in liste_matrice_alien: # On itére les matrices à traiter une par une.

        for lettre_alien in range(65, 74): # On itère de 65 à 74 (convertit en ASCII, cela nous donne A,B,...,I) pour prendre UNE matrice référence parmis notre dictionnaire de matrices références.

            marqueur = True # Le marqueur commence à True et le restera si les matrices sont identiques.

            for i, ligne in enumerate(reference[chr(lettre_alien)]): # On récupère la ligne dans la matrice référence d'un alien et son indice.
                for j, valeur in enumerate(ligne): # On récupère une valeur (colone) dans ligne de la matrice référence d'un alien et son indice.

                    if valeur != matrice_alien[i][j]: # On compare la valeur récupéré à la valeur, au même indice, dans la matrice de l'alien en traitement.
                        marqueur = False # Si les valeurs sont différentes, on passe le marqueur à False (Les matrices ne sont pas identiques).

            if marqueur: # Si le marqueur est resté True, la matrice de l'alien traité est identique à la matrice référence actuellement utilisé.
                resultat += chr(lettre_alien) # On ajoute la lettre de l'alien au résultat.
                
    return(resultat) # On renvoit le résultat.

def main():

    # Boucle while qui réessaye de récuperer l'image si echec.
    while True:
        # Connexion au défi et récupération des entrées:
        try:
            d = DefisUrl('https://pydefis.callicode.fr/defis/C22_Invaders01/get/cityhand/48ad5', verify=True)
            lignes = d.get()
            str_orig = "".join(lignes) # Entrées stocké sous forme de chaîne de caractère (Image codée en base 64).

            #Décodage de l'image avec les librairies 'io' et 'base64': 
            image_pil = Image.open(BytesIO(base64.b64decode(str_orig))) # L'entrée devient une image format PNG stocké dans 'image_pil' grâce à la librairie 'pillow'.
            break # Si tout s'est bien passé, on quitte la boucle while.
        except:
            sleep(5) # Si erreur pendant la récupération des entrées, on attend 5 secondes et on recommence
    
    with open("reference.json", "r") as file: # On ouvre le fichier json contenant le dictionnaire référençant les aliens (clé = Lettre, valeur = Matrice).
        reference = json.load(file) # On stock le dictionnaire dans la variable 'reference'.

    res = comparer(reference, decomposition(image_pil)) # Appelle des fonctions pour trouver la réponse :

    #Affichage de la réponse pour contrôle et envoi vers le site pydefis.fr :
    print(res)
    rep = d.post(res)
    print(rep)


main() # Lancement du programme

EBBAAEBADBEAADCEBDCEEAAB
Bravo, défi résolu !!!
